In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name
%load_ext jupyter_black
spark = SparkSession.builder.appName("project1").getOrCreate()

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black
23/01/08 22:45:54 WARN Utils: Your hostname, zechchair-gl553vd resolves to a loopback address: 127.0.1.1; using 192.168.0.130 instead (on interface wlp2s0)
23/01/08 22:45:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/08 22:45:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [46]:
from os import listdir
from os.path import isfile, join

my_path = "/home/zechchair/Documents/github/M2-data-science/big data /project/input"
only_files = [my_path + "/" + f for f in listdir(my_path) if isfile(join(my_path, f))]
files_names = [f for f in listdir(my_path) if isfile(join(my_path, f))]
onlyCities_names = set(
    [files_name.split(".")[0].split("_")[0] for files_name in files_names]
)

In [125]:
from pyspark.sql.functions import split, reverse, when, lit, avg, sum

In [141]:
df = spark.read.format("csv").load(my_path + "/*.txt")

# Add a column with the file name
df = df.withColumn("city", input_file_name())
df = df.withColumn("store_N", reverse(split("city", "/"))[0])
df = df.withColumn("city", split("store_N", "\\.")[0])
df = df.withColumn("city", split("city", "_")[0])
df = df.withColumn("income", df["_c0"].substr(5, 7))
df = df.withColumn("month", df["_c0"].substr(0, 3))
df.show()

+------+----------+----------------+------+-----+
|   _c0|      city|         store_N|income|month|
+------+----------+----------------+------+-----+
|JAN 13|      lyon|        lyon.txt|    13|  JAN|
|FEB 12|      lyon|        lyon.txt|    12|  FEB|
|MAR 14|      lyon|        lyon.txt|    14|  MAR|
|APR 15|      lyon|        lyon.txt|    15|  APR|
|MAY 12|      lyon|        lyon.txt|    12|  MAY|
|JUN 15|      lyon|        lyon.txt|    15|  JUN|
|JUL 19|      lyon|        lyon.txt|    19|  JUL|
|AUG 25|      lyon|        lyon.txt|    25|  AUG|
|SEP 13|      lyon|        lyon.txt|    13|  SEP|
|OCT 11|      lyon|        lyon.txt|    11|  OCT|
|NOV 22|      lyon|        lyon.txt|    22|  NOV|
|DEC 22|      lyon|        lyon.txt|    22|  DEC|
|JAN 21|marseilles|marseilles_1.txt|    21|  JAN|
|FEB 21|marseilles|marseilles_1.txt|    21|  FEB|
|MAR 21|marseilles|marseilles_1.txt|    21|  MAR|
|APR 27|marseilles|marseilles_1.txt|    27|  APR|
|MAY 25|marseilles|marseilles_1.txt|    25|  MAY|


In [142]:
df.select(avg("income")).show()

+-----------------+
|      avg(income)|
+-----------------+
|23.19871794871795|
+-----------------+



In [143]:
df.groupBy("month").agg(avg("income")).show()

+-----+------------------+
|month|       avg(income)|
+-----+------------------+
|  APR| 20.23076923076923|
|  OCT| 26.53846153846154|
|  NOV| 24.53846153846154|
|  FEB|19.153846153846153|
|  SEP| 25.53846153846154|
|  JAN| 20.76923076923077|
|  AUG|23.076923076923077|
|  MAR| 17.53846153846154|
|  DEC|              29.0|
|  JUN|27.846153846153847|
|  JUL|21.692307692307693|
|  MAY| 22.46153846153846|
+-----+------------------+



In [144]:
df.groupBy("city", "month").agg(avg("income")).orderBy("city").show()

+-----+-----+-----------+
| city|month|avg(income)|
+-----+-----+-----------+
|anger|  AUG|       15.0|
|anger|  MAY|       12.0|
|anger|  SEP|       13.0|
|anger|  APR|       15.0|
|anger|  MAR|       14.0|
|anger|  NOV|       14.0|
|anger|  JAN|       13.0|
|anger|  JUN|       15.0|
|anger|  JUL|       19.0|
|anger|  DEC|       16.0|
|anger|  OCT|        8.0|
|anger|  FEB|       12.0|
| lyon|  MAR|       14.0|
| lyon|  AUG|       25.0|
| lyon|  APR|       15.0|
| lyon|  MAY|       12.0|
| lyon|  OCT|       11.0|
| lyon|  JUL|       19.0|
| lyon|  FEB|       12.0|
| lyon|  JUN|       15.0|
+-----+-----+-----------+
only showing top 20 rows



In [145]:
df.groupBy("city").agg(avg("income")).orderBy("city").show()

+----------+------------------+
|      city|       avg(income)|
+----------+------------------+
|     anger|13.833333333333334|
|      lyon|16.083333333333332|
|marseilles|21.458333333333332|
|    nantes|             17.25|
|      nice|16.916666666666668|
|    orlean|16.333333333333332|
|     paris| 43.55555555555556|
|    rennes|              15.0|
|  toulouse|             14.75|
|    troyes|17.833333333333332|
+----------+------------------+



In [146]:
df.groupBy("city").agg(avg("income")).orderBy("city").show()

+----------+------------------+
|      city|       avg(income)|
+----------+------------------+
|     anger|13.833333333333334|
|      lyon|16.083333333333332|
|marseilles|21.458333333333332|
|    nantes|             17.25|
|      nice|16.916666666666668|
|    orlean|16.333333333333332|
|     paris| 43.55555555555556|
|    rennes|              15.0|
|  toulouse|             14.75|
|    troyes|17.833333333333332|
+----------+------------------+



In [147]:
df.groupBy("month", "store_N").agg(sum("income")).show()

+-----+----------------+-----------+
|month|         store_N|sum(income)|
+-----+----------------+-----------+
|  DEC|        lyon.txt|       22.0|
|  SEP|marseilles_1.txt|       23.0|
|  OCT|        lyon.txt|       11.0|
|  MAY|        lyon.txt|       12.0|
|  JAN|marseilles_1.txt|       21.0|
|  FEB|marseilles_1.txt|       21.0|
|  APR|        lyon.txt|       15.0|
|  MAR|marseilles_1.txt|       21.0|
|  APR|marseilles_1.txt|       27.0|
|  AUG|marseilles_1.txt|       22.0|
|  OCT|marseilles_1.txt|       28.0|
|  NOV|        lyon.txt|       22.0|
|  JUL|        lyon.txt|       19.0|
|  JUN|        lyon.txt|       15.0|
|  JUL|marseilles_1.txt|       21.0|
|  JUN|marseilles_1.txt|       25.0|
|  MAR|        lyon.txt|       14.0|
|  MAY|marseilles_1.txt|       25.0|
|  NOV|marseilles_1.txt|       24.0|
|  DEC|marseilles_1.txt|       26.0|
+-----+----------------+-----------+
only showing top 20 rows



In [148]:
df.groupBy("month", "store_N").agg(avg("income")).show()

+-----+----------------+-----------+
|month|         store_N|avg(income)|
+-----+----------------+-----------+
|  DEC|        lyon.txt|       22.0|
|  SEP|marseilles_1.txt|       23.0|
|  OCT|        lyon.txt|       11.0|
|  MAY|        lyon.txt|       12.0|
|  JAN|marseilles_1.txt|       21.0|
|  FEB|marseilles_1.txt|       21.0|
|  APR|        lyon.txt|       15.0|
|  MAR|marseilles_1.txt|       21.0|
|  APR|marseilles_1.txt|       27.0|
|  AUG|marseilles_1.txt|       22.0|
|  OCT|marseilles_1.txt|       28.0|
|  NOV|        lyon.txt|       22.0|
|  JUL|        lyon.txt|       19.0|
|  JUN|        lyon.txt|       15.0|
|  JUL|marseilles_1.txt|       21.0|
|  JUN|marseilles_1.txt|       25.0|
|  MAR|        lyon.txt|       14.0|
|  MAY|marseilles_1.txt|       25.0|
|  NOV|marseilles_1.txt|       24.0|
|  DEC|marseilles_1.txt|       26.0|
+-----+----------------+-----------+
only showing top 20 rows



In [160]:
from pyspark.sql.functions import split, reverse, when, lit, avg, sum, max, col

# Group the data by the month column
grouped_df = df.groupBy("month")

# Find the maximum income for each group
max_df = grouped_df.agg(max(col("income")).alias("max_income"))
max_df = max_df.withColumnRenamed("month", "Month_max")


# Join the original data with the aggregated data on the month and income columns
joined_df = df.join(
    max_df,
    (df["month"] == max_df["Month_max"]) & (df["income"] == max_df["max_income"]),
)

# Select the month, store, and max_income columns
result_df = joined_df.select("month", "store_N", "max_income")
result_df.show()

+-----+-----------+----------+
|month|    store_N|max_income|
+-----+-----------+----------+
|  APR| rennes.txt|         9|
|  APR|   nice.txt|         9|
|  AUG|paris_2.txt|        45|
|  DEC|paris_1.txt|        71|
|  FEB|paris_2.txt|        42|
|  JAN|paris_1.txt|        51|
|  JUL|paris_1.txt|        61|
|  JUN|paris_2.txt|        85|
|  MAR|paris_2.txt|        44|
|  MAY|paris_2.txt|        72|
|  NOV|paris_2.txt|        64|
|  OCT| orlean.txt|         8|
|  OCT|  anger.txt|         8|
|  SEP|paris_2.txt|        63|
+-----+-----------+----------+

